In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/inputdata/new_data1.json
/kaggle/input/inputdata/arabic_conversation.json
/kaggle/input/inputdata/arabic_conversation_simulation.json
/kaggle/input/inputdata/ChatData.py


In [2]:
!pip install transformers
!pip install rouge
!pip install pytorch-lamb

In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.optim import Adam
from torch.utils.data import DataLoader
import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import json
import random
from rouge import Rouge 
from pytorch_lamb import Lamb



/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [4]:

def calculate_rouge(hypothesis, reference):
    rouge = Rouge()
    scores = rouge.get_scores(hypothesis, reference)
    return scores


In [5]:
def train(path, model, optim,epochs):
    chatData = ChatData(path, tokenizer)
    chatData =  DataLoader(chatData, num_workers=2, batch_size=200)
    test = ("<startofstring>"+"<Q>مرحبًا<bot>:")
    output_referance =infer(test)
    model.train()

    

    print("training .... ")
    
    for i in tqdm.tqdm(range(epochs)):
        for X, a,Y in chatData:
            X = X.to(device)
            a = a.to(device)
            Y = Y.to(device)
            optim.zero_grad()
            #loss = model(X, attention_mask=a, labels=X).loss
            losses = model(X, attention_mask=a, labels=Y).loss
            loss = losses.mean()
            loss.backward()
            optim.step()
        # Save the model after each epoch
        torch.save(model.state_dict(), f"model_state_epoch_{i}.pt")

        # Evaluation...
        model.eval()
        with torch.no_grad():
            output = infer(test)
            print(output)
            print(calculate_rouge(output, output_referance))
        model.train()
def infer(inp):
    inp = tokenizer(inp, return_tensors="pt")
    X = inp["input_ids"].to(device)
    a = inp["attention_mask"].to(device)
    output =model.module.generate(X, attention_mask=a,max_new_tokens=50 
                        ,pad_token_id=tokenizer.pad_token_id,eos_token_id=tokenizer.eos_token_id
                           )
    output = tokenizer.decode(output[0])
    return output


In [6]:

class ChatData(Dataset):
    def __init__(self, path:str, tokenizer):
        self.data = json.load(open(path, "r", encoding="utf-8"))
        self.X = []
        self.Y = []
        tmp = ""
        for i in self.data:
            for j in i['dialog']:
                if j['sender'] == "participant2":
                    self.X.append("<startofstring>"+j['text'])
                    tmp = "<startofstring>"+j['text']+"<bot>:"
                else:
                    self.Y.append(tmp+j['text']+"<endofstring>")
                    tmp = ""

        #combined = list(zip(self.X, self.Y))
        #random.shuffle(combined)

        #self.X[:], self.Y[:] = zip(*combined)

        self.X = self.X[:10000]
        self.Y = self.Y[:10000]
        print(self.Y[0])
        self.X_encoded = tokenizer(self.X, max_length=40, truncation=True, padding="max_length", return_tensors="pt")
        self.Y_encoded = tokenizer(self.Y, max_length=40, truncation=True, padding="max_length", return_tensors="pt")

        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']

        self.target_ids = self.Y_encoded['input_ids']
        
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attention_mask[idx], self.target_ids[idx]


In [7]:
model_name = "aubmindlab/aragpt2-base"
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens({"pad_token": "<pad>", 
                                "bos_token": "<startofstring>",
                                "eos_token": "<endofstring>"})

tokenizer.add_tokens(["<bot>:","<prevQ>","<prevA>","<Q>"])
model = GPT2LMHeadModel.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)
model = model.to(device)
#optim = Lamb(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-6, weight_decay=0.01)

optim = Adam(model.parameters(), lr=1e-3)



Let's use 2 GPUs!


In [ ]:
train("/kaggle/input/inputdata/new_data1.json", model, optim,10)

<startofstring><Q>مرحبًا<bot>:مرحبًا بك في مؤسسة اكسبو شوب! كيف يمكنني مساعدتك؟<endofstring>
training .... 


  0%|          | 0/10 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
 10%|█         | 1/10 [01:08<10:20, 68.94s/it]

<startofstring> <Q> مرحبًا <bot>: مرحبًا بك في مؤسسة اكسبو شوب! كيف يمكنني مساعدتك؟ <prevA> ارغب بالاستفسار عن ساعة ذكية. هل يمكنك مساعدتي؟ <prevA> لدينا عرض <prevA> لدينا عرض <bot>: لدينا عرض <bot>: لدينا عرض <prevA> لد
[{'rouge-1': {'r': 1.0, 'p': 0.16666666666666666, 'f': 0.2857142832653061}, 'rouge-2': {'r': 1.0, 'p': 0.10714285714285714, 'f': 0.19354838534859523}, 'rouge-l': {'r': 1.0, 'p': 0.16666666666666666, 'f': 0.2857142832653061}}]


In [ ]:
print("infer from model : ")
prevQ = ""
prevA = ""
while True:
  inp = input()
  if inp == "-1":
    break;
  
  inp = "<startofstring>"+prevQ+prevA+"<Q>"+inp+"<bot>:"
  prevQ = "<prevQ>"+inp
  output = infer(inp)
  prevA = "<prevA>"+output
  prevA.replace("<bot>:","")
  print(output)